Khai báo các thư vienj cần thiết

In [101]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.decomposition import PCA as sklearnPCA
from sklearn import datasets
from sklearn.model_selection import train_test_split
from scipy import sparse

Đọc dữ liệu và chia dữ liệu theo tỉ lệ: train:validation = 4:1

In [102]:
iris = datasets.load_iris()
X = iris.data[:, :4]
Y = iris.target
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

Các phương thức cần thiết cho mô hình hồi quy Logistic nhiều lớp

In [103]:
C = 3
def convert_labels(y, C = C):
    """
    convert 1d label to a matrix label: each column of this
    matrix coresponding to 1 element in y. In i-th column of Y,
    only one non-zeros element located in the y[i]-th position,
    and = 1 ex: y = [0, 2, 1, 0], and 3 classes then return

            [[1, 0, 0, 1],
             [0, 0, 1, 0],
             [0, 1, 0, 0]]
    """
    Y = sparse.coo_matrix((np.ones_like(y),
        (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y


def softmax_stable(Z):
    """
    Compute softmax values for each sets of scores in Z.
    each column of Z is a set of score.
    """
    e_Z = np.exp(Z - np.max(Z, axis = 0, keepdims = True))
    A = e_Z / e_Z.sum(axis = 0)
    return A

def softmax(Z):
    """
    #Compute softmax values for each sets of scores in V.
    #each column of V is a set of score.
    """
    e_Z = np.exp(Z)
    A = e_Z / e_Z.sum(axis = 0)
    return A

def softmax_regression(X, y, W_init, eta, tol = 1e-4, max_count = 10000):
    W = [W_init]
    C = W_init.shape[1]
    Y = convert_labels(y, C)
    it = 0
    N = X.shape[1]
    d = X.shape[0]

    count = 0
    check_w_after = 20
    while count < max_count:
        # mix data
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = Y[:, i].reshape(C, 1)
            ai = softmax(np.dot(W[-1].T, xi))
            W_new = W[-1] + eta*xi.dot((yi - ai).T)
            count += 1
            # stopping criteria
            if count%check_w_after == 0:
                if np.linalg.norm(W_new - W[-check_w_after]) < tol:
                    return W
            W.append(W_new)
    return W

# cost or loss function
def cost(X, Y, W):
    A = softmax(W.T.dot(X))
    return -np.sum(Y*np.log(A))

# Predict that X belong to which class (1..C now indexed as 0..C-1 )
def pred(W, X):
    """
    predict output of each columns of X
    Class of each x_i is determined by location of max probability
    Note that class are indexed by [0, 1, 2, ...., C-1]
    """
    A = softmax_stable(W.T.dot(X))
    return np.argmax(A, axis = 0)



In ra bộ hệ số của mô hình 

In [121]:
one_train = np.ones((X_train.shape[0], 1))
Xbar_train = np.concatenate((one_train, X_train), axis = 1).T

eta = .05
d = X.T.shape[0]
W_init = np.random.randn(Xbar_train.shape[0], C)
W = softmax_regression(Xbar_train, Y_train, W_init, eta)
print(W[-1])

[[-0.73871626  2.21402564 -5.0062918 ]
 [ 2.45825632  0.90897683 -3.15016637]
 [ 2.40372996  0.04267035 -5.23905817]
 [-5.76619147 -1.29183113  5.97062684]
 [-2.90128806 -2.88137854  5.18987667]]


Dự đoán dữ liệu

In [125]:
one_test = np.ones((X_test.shape[0], 1))
Xbar_test = np.concatenate((one_test, X_test), axis = 1).T
y_predict = pred(W[-1], Xbar_test)
print(y_predict)

[1 0 2 1 1 0 1 2 2 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
